### 자동차 번호판 인식 - CRNN 구현
#### 구조 구현 확인 (CNN 구조/ RNN 구조 확인) - o
#### **loss 선언 오류 (blank/zero_infinity)** - 여기서 에러
#### augmentation 영향 확인
#### learning rate/ gradient clip 영향 확인



### github 연동

In [56]:
cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [57]:
!git add crnn.ipynb

fatal: not a git repository (or any parent up to mount point /content)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


### data unzip

In [1]:
# %cd /content/drive/MyDrive/Colab Notebooks/data/train

# !unzip -qq "/content/drive/MyDrive/Colab Notebooks/data/train/[원천]자동차번호판OCR데이터.zip"

In [2]:
# %cd /content/drive/MyDrive/Colab Notebooks/data/val

# !unzip -qq "/content/drive/MyDrive/Colab Notebooks/data/val/[원천]자동차번호판OCR데이터.zip"

In [3]:
# import os

In [4]:
# train_list = os.listdir('/content/drive/MyDrive/Colab Notebooks/data/train')
# train_list = [name for name in train_list if name.endswith('.jpg')]
# train_list[:10]

In [5]:
# val_list = os.listdir('/content/drive/MyDrive/Colab Notebooks/data/val')
# val_list = [name for name in val_list if name.endswith('.jpg')]
# val_list[:10]

In [6]:
# len(train_list), len(val_list)

#### pickle로 저장

In [7]:
import pickle
import numpy as np

from PIL import Image

In [8]:
def make_pickle(data_dir, name_list):
  X_list, y_list = [], []
  for img_name in name_list:
    img_path = os.path.join(data_dir, img_name)
    image = Image.open(img_path).convert('L')
    image = image.resize((100, 32))
    label = img_name.split('.')[0].split('-')[0]
    X_list.append(image)
    y_list.append(label)

  with open(f'{data_dir}.pickle', 'wb') as f:
    pickle.dump([np.array(X_list), np.array(y_list)], f)

In [9]:
# train_list = os.listdir('/content/drive/MyDrive/Colab Notebooks/data/train')
# train_list = [name for name in train_list if name.endswith('.jpg')]

# make_pickle('/content/drive/MyDrive/Colab Notebooks/data/train', train_list)

In [10]:
# val_list = os.listdir('/content/drive/MyDrive/Colab Notebooks/data/val')
# val_list = [name for name in val_list if name.endswith('.jpg')]

# make_pickle('/content/drive/MyDrive/Colab Notebooks/data/val', val_list)

In [11]:
# len(val_list)

### 패키지 임포트

In [12]:
import os
from tqdm import tqdm

from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader

In [13]:
import unicodedata

def trans(x):
  return unicodedata.normalize('NFC', x)

### 토크나이저 정의

In [14]:
# tokenizer = {'-':0, '0':1, '1':2, '2':3, '3':4, '4':5, '5':6, '6':7, '7':8, '8':9, '9':10,} ######### '-' 추가
# idx = 11
# for name in train_list+val_list:
#   name = trans(name.split('.')[0].split('-')[0])
#   for s in name:
#     if s not in tokenizer:
#       tokenizer[s] = idx
#       idx += 1

In [15]:
# # val_list
# print(tokenizer)
# print(len(tokenizer))

In [16]:
# print(tokenizer)

In [17]:
import json

In [18]:
# with open(f'/content/drive/MyDrive/Colab Notebooks/data/tokenizer.json', 'w') as f:
#   json.dump(tokenizer, f)

### 데이터셋/모델 정의

In [19]:
class NumberPlateDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        image, label = self.X[idx], self.y[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [20]:
transform = transforms.Compose([
    # transforms.Resize((32, 100)), # h, w
    transforms.ToTensor(), # c, h, w
    transforms.Normalize(mean=[0.5], std=[0.5])
])

In [21]:
class CRNN(nn.Module):
    def __init__(self, num_classes, hidden_size):
        super(CRNN, self).__init__()
        self.convolutional_layers = nn.Sequential(
            nn.Conv2d(1, 64, (3, 3), (1, 1), (1, 1), bias=True),
            nn.ReLU(True),
            nn.MaxPool2d((2, 2), (2, 2)),  # image size: 16 * 64

            nn.Conv2d(64, 128, (3, 3), (1, 1), (1, 1), bias=True),
            nn.ReLU(True),
            nn.MaxPool2d((2, 2), (2, 2)),  # image size: 8 * 32

            nn.Conv2d(128, 256, (3, 3), (1, 1), (1, 1), bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.Conv2d(256, 256, (3, 3), (1, 1), (1, 1), bias=True),
            nn.ReLU(True),
            nn.MaxPool2d((2, 2), (2, 1), (0, 1)),  # image size: 4 x 16

            nn.Conv2d(256, 512, (3, 3), (1, 1), (1, 1), bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.Conv2d(512, 512, (3, 3), (1, 1), (1, 1), bias=True),
            nn.ReLU(True),
            nn.MaxPool2d((2, 2), (2, 1), (0, 1)),  # image size: 2 x 16

            nn.Conv2d(512, 512, (2, 2), (1, 1), (0, 0), bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),  # image size: 1 x 16
        )

        self.rnn1 = nn.LSTM(512, hidden_size, bidirectional=True)
        # self.rnn2 = nn.LSTM(512, hidden_size, bidirectional=True)

        self.output = nn.Linear(hidden_size * 2, num_classes)

    def forward(self, x):
        x = self.convolutional_layers(x)  # [b, c, h, w]
        x = x.squeeze(2) # [b, c, w]
        x = x.permute(2, 0, 1) # [w, b, c]
        # print(x.shape)

        x, _ = self.rnn1(x) # [w, b, hidden*2]
        # x, _ = self.rnn2(x)

        sequence_length, batch_size, inputs_size = x.size()
        x = x.view(sequence_length * batch_size, inputs_size) # [w*b, hidden*2]

        x = self.output(x) # [w*b, c]
        x = x.view(sequence_length, batch_size, -1) # [w, b, c]
        return x

In [22]:
with open('/content/drive/MyDrive/Colab Notebooks/data/train.pickle', 'rb') as f:
  X, y = pickle.load(f)

train_dataset = NumberPlateDataset(X, y, transform=transform) # c, h, w
train_loader = DataLoader(train_dataset, batch_size=128, shuffle = True)

In [23]:
train_dataset[0][0].shape

torch.Size([1, 32, 100])

In [24]:
len(train_dataset)

80000

In [42]:
with open('/content/drive/MyDrive/Colab Notebooks/data/val.pickle', 'rb') as f:
  X, y = pickle.load(f)

val_dataset = NumberPlateDataset(X, y, transform=transform) # c, h, w
val_loader = DataLoader(val_dataset, batch_size=128, shuffle = False)

In [26]:
# val_dataset = NumberPlateDataset(data_dir='/content/drive/MyDrive/Colab Notebooks/data/val', transform=transform)
# val_loader = DataLoader(val_dataset, batch_size=512, shuffle = True)

In [27]:
with open(f'/content/drive/MyDrive/Colab Notebooks/data/tokenizer.json', 'r') as f:
  tokenizer = json.load(f)

In [28]:
def encode_label(label, tokenizer):
  encode = []
  for s in label:
    encode.append(tokenizer[s])

  return tuple(encode)

def encode_text_batch(labels):
  length_batch = tuple()
  encode_batch = tuple()
  for label in labels:
    label = trans(label)
    encode = encode_label(label, tokenizer)
    encode_batch += (encode, )

    length_batch += (len(label), )

  return encode_batch, length_batch

### 학습

In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CRNN(99, 256).to(device)
criterion = nn.CTCLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [30]:
model_path = '/content/drive/MyDrive/Colab Notebooks/'

In [31]:
tokenizer_reverse = {v: k for k, v in tokenizer.items()}

print(tokenizer_reverse)

def decode_pred(pred, tokenizer): # tokenizer_reverse
  decode = []
  for i in range(len(pred)):
    if i > 0 and pred[i] == pred[i-1]:
      continue
    if pred[i] == 0 or pred[i] == 97: ######## 여기 에러 있었음
      continue
    decode.append(tokenizer[pred[i]])

  return decode

def decode_batch(outputs, tokenizer): # tokenizer_reverse
  batch_decode = []
  for pred in outputs:
    decode = decode_pred(pred, tokenizer)
    batch_decode.append(decode)

  return batch_decode

{0: '-', 1: '0', 2: '1', 3: '2', 4: '3', 5: '4', 6: '5', 7: '6', 8: '7', 9: '8', 10: '9', 11: '가', 12: '거', 13: '고', 14: '구', 15: '나', 16: '너', 17: '노', 18: '누', 19: '다', 20: '더', 21: '도', 22: '두', 23: '라', 24: '러', 25: '로', 26: '루', 27: '마', 28: '머', 29: '모', 30: '무', 31: '버', 32: '보', 33: '부', 34: '서', 35: '소', 36: '수', 37: '어', 38: '오', 39: '우', 40: '저', 41: '조', 42: '주', 43: '하', 44: '허', 45: '호', 46: '자', 47: '이', 48: '바', 49: '아', 50: '리', 51: '육', 52: '히', 53: '사', 54: '강', 55: '원', 56: '경', 57: '기', 58: '배', 59: '지', 60: '양', 61: '타', 62: '광', 63: '명', 64: '파', 65: '김', 66: '포', 67: '천', 68: '차', 69: '카', 70: '안', 71: '산', 72: '인', 73: '남', 74: '북', 75: '대', 76: '전', 77: '울', 78: '관', 79: '악', 80: '금', 81: '동', 82: '문', 83: '작', 84: '초', 85: '영', 86: '등', 87: '용', 88: '뱌', 89: '계', 90: '미', 91: '홀', 92: '추', 93: '평', 94: '연', 95: '중', 96: '충', 97: ' ', 98: '시'}


In [32]:
max_epoch = 200
patience = 5

In [33]:
val_log = np.zeros(max_epoch+1)

for epoch in range(1, max_epoch+1):
    model.train()

    correct_predictions, total_predictions = 0, 0
    for images, labels in tqdm(train_loader, desc = f"Epoch:{epoch}"):
        images = images.to(device)

        # print(images.shape)
        labels_encoded, label_lengths = encode_text_batch(labels)

        labels_encoded_tensor = torch.cat([torch.tensor(label, dtype=torch.long) for label in labels_encoded])
        label_lengths_tensor = torch.tensor(label_lengths, dtype=torch.long)

        outputs = model(images)
        outputs = F.log_softmax(outputs, 2)

        images_lengths = torch.LongTensor([outputs.size(0)] * outputs.size(1))

        # Computational loss
        loss = criterion(outputs, labels_encoded_tensor, images_lengths, label_lengths_tensor)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        preds = outputs.argmax(2)
        preds = preds.permute(1, 0)
        # print(preds.shape)

        # print(labels)

        decodes = decode_batch(preds.cpu().numpy(), tokenizer_reverse)
        for i in range(len(decodes)):
          if ''.join(decodes[i]) == labels[i]:
            correct_predictions += 1
          total_predictions += 1

    train_accuracy = correct_predictions / total_predictions

    print(f'[Train] Epoch {epoch}, Loss: {loss.item():.2f}, Accuracy : {train_accuracy*100:.2f}')

    with torch.no_grad():
      model.eval()

      correct_predictions, total_predictions = 0, 0
      for images, labels in tqdm(val_loader, desc = f"epoch:{epoch}"):
        images = images.to(device)

        labels_encoded, label_lengths = encode_text_batch(labels)

        labels_encoded_tensor = torch.cat([torch.tensor(label, dtype=torch.long) for label in labels_encoded])
        label_lengths_tensor = torch.tensor(label_lengths, dtype=torch.long)

        outputs = model(images)
        outputs = F.log_softmax(outputs, 2)

        images_lengths = torch.LongTensor([outputs.size(0)] * outputs.size(1))

        # Computational loss
        loss = criterion(outputs, labels_encoded_tensor, images_lengths, label_lengths_tensor)

        preds = outputs.argmax(2)
        preds = preds.permute(1, 0)

        decodes = decode_batch(preds.cpu().numpy(), tokenizer_reverse)
        for i in range(len(decodes)):
          if ''.join(decodes[i]) == labels[i]:
            correct_predictions += 1
          total_predictions += 1

    val_accuracy = correct_predictions / total_predictions

    print(f'[Val] Epoch {epoch}, Loss: {loss.item():.2f}, Accuracy : {val_accuracy*100:.2f}')

    val_log[epoch] = - val_accuracy

    if np.argmin(val_log[:epoch + 1]) == epoch:
        torch.save(model.state_dict(), f'{model_path}{epoch}.pt')

    elif np.argmin(val_log[:epoch + 1]) <= epoch - patience:
        break


Epoch:1: 100%|██████████| 625/625 [01:15<00:00,  8.27it/s]


[Train] Epoch 1, Loss: 1.98, Accuracy : 0.00


epoch:1: 100%|██████████| 79/79 [00:04<00:00, 17.67it/s]


[Val] Epoch 1, Loss: 2.14, Accuracy : 0.01


Epoch:2: 100%|██████████| 625/625 [01:15<00:00,  8.33it/s]


...

[Val] Epoch 19, Loss: 0.00, Accuracy : 91.31


Epoch:20: 100%|██████████| 625/625 [01:15<00:00,  8.33it/s]


[Train] Epoch 20, Loss: 0.02, Accuracy : 97.14


epoch:20: 100%|██████████| 79/79 [00:04<00:00, 17.78it/s]

[Val] Epoch 20, Loss: 0.28, Accuracy : 91.12


### 평가

In [35]:
model_path = '/content/drive/MyDrive/Colab Notebooks/15.pt'

model = CRNN(99, 256).to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [43]:
with torch.no_grad():
  model.eval()

  correct_predictions, total_predictions = 0, 0
  eval_preds = []
  for images, labels in tqdm(val_loader, desc = f"final eval"):
    images = images.to(device)

    labels_encoded, label_lengths = encode_text_batch(labels)

    labels_encoded_tensor = torch.cat([torch.tensor(label, dtype=torch.long) for label in labels_encoded])
    label_lengths_tensor = torch.tensor(label_lengths, dtype=torch.long)

    outputs = model(images)
    outputs = F.log_softmax(outputs, 2)

    images_lengths = torch.LongTensor([outputs.size(0)] * outputs.size(1))

    # Computational loss
    loss = criterion(outputs, labels_encoded_tensor, images_lengths, label_lengths_tensor)

    preds = outputs.argmax(2)
    preds = preds.permute(1, 0)

    decodes = decode_batch(preds.cpu().numpy(), tokenizer_reverse)
    for i in range(len(decodes)):
      if ''.join(decodes[i]) == labels[i]:
        correct_predictions += 1
      total_predictions += 1
      eval_preds.append(''.join(decodes[i]))

val_accuracy = correct_predictions / total_predictions

final eval: 100%|██████████| 79/79 [00:04<00:00, 17.38it/s]


In [44]:
print(f'eval Accuracy : {val_accuracy*100:.2f}')

eval Accuracy : 91.46


In [45]:
print(eval_preds)

['01가3733', '01가0598', '01거0608', '01고8109', '08구8385', '01너0665', '01노5511', '01다1275', '01머7076', '01도8490', '01도9251', '01두1316', '01두8564', '01라8128', '01러3672', '01로1282', '01로5378', '01루0999', '01루0999', '01마3235', '01마8037', '01모3121', '01모3154', '01모3154', '01무0607', '01무4621', '01보9398', '01보9445', '51보4353', '01수2431', '01수7052', '01수7052', '01어2461', '01우6981', '01우9493', '01우9493', '01저0754', '01조0261', '01주6939', '01주6939', '01주6939', '01하1096', '01허7049', '02거2362', '02거3803', '02고7956', '02고8360', '02나0948', '02나5269', '02너0512', '02노0270', '08수3454', '02누1781', '02누5790', '02더4137', '02두0820', '02두0820', '02라3170', '02라3170', '02라3170', '02라3170', '02라3170', '02라3170', '02라4201', '02라8214', '02로6470', '06노8771', '02마9543', '02모5433', '02무8772', '02버1855', '02보0665', '02보7765', '02부6807', '02부7482', '02부9180', '02소1888', '02수8090', '02수8559', '02어5556', '92라9212', '02오8951', '02오8951', '03마5884', '02주0141', '02주2593', '02어9354', '02호8290', '03가7827', '03거6147', '03저6164'